In [17]:
!pip install --upgrade langchain
!pip install --upgrade pypdfium2
!pip install easyocr
!pip install --upgrade langchain-huggingface
!pip install --upgrade huggingface_hub
!pip install transformers
!pip install accelerate
!pip install  bitsandbytes

In [30]:
from google.colab import userdata
HUGGINGFACEHUB_API_TOKEN = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [31]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

In [37]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [38]:
import os
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [4]:
from easyocr import Reader

# Load model for the English language
language_reader = Reader(["en"])

In [5]:
import pypdfium2 as pdfium
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

In [6]:
def convert_pdf_to_images(file_path, scale=300/72):

    pdf_file = pdfium.PdfDocument(file_path)
    page_indices = [i for i in range(len(pdf_file))]

    renderer = pdf_file.render(
        pdfium.PdfBitmap.to_pil,
        page_indices = page_indices,
        scale = scale,
    )

    list_final_images = []

    for i, image in zip(page_indices, renderer):

        image_byte_array = BytesIO()
        image.save(image_byte_array, format='jpeg', optimize=True)
        image_byte_array = image_byte_array.getvalue()
        list_final_images.append(dict({i:image_byte_array}))

    return list_final_images

In [7]:
convert_pdf_to_images = convert_pdf_to_images('/content/medical_report.pdf')

In [8]:
def extract_text_with_easyocr(list_dict_final_images):

    image_list = [list(data.values())[0] for data in list_dict_final_images]
    image_content = []

    for index, image_bytes in enumerate(image_list):

        image = Image.open(BytesIO(image_bytes))
        raw_text = language_reader.readtext(image)
        raw_text = "\n".join([res[1] for res in raw_text])

        image_content.append(raw_text)

    return "\n".join(image_content)

In [9]:
text_with_easy_ocr = extract_text_with_easyocr(convert_pdf_to_images)
print(text_with_easy_ocr)

Medical Record
1
Patient Information
Name: Vikram Singh
Age: 60
Gender: Male
Date of Birth: March 15, 1964
Aadhaar Card Number: 1234-5678-9012
Residential Address: 45, Green Park Colony, Delhi, 110016
PAN Card Number: ABCPS1234E
Company: ABC Technologies Pvt: Ltd.
Insurance ID: INS-987654321
Insurance Company: HealthSecure Insurance Co.
Insurance Type: Comprehensive Health Plan
Hospital Information
Hospital Name: City Care Hospital
Address: 123, Main Road, Delhi, 110001
Phone: +91 11 2345 6789
Email: info@citycarehospital.com
Medical History
6/10
KnownConditions:
ChronicKidney Disea se ( C KD ) , Stage 5 (End- Stage RenalDisea s e-ES
RD )
Ayperten sio n
Type 2 Diabete s Mellitu s
Alergie s:
Sulfadrug $
Nuts
CurrentMedication s:
Amlodipine 5m gdaily ( forhyperten sion )
Metformin5 0 0 mgtwicedaily (fordiab ete s )
Calciumcarb onate (p ho sphatebinder)
Epoetinalfainjection s (W eeklyforanemia )
Past Surgerie s orMajorTreatments:
Fistulacreationfordialy sis ( 2 0 2 2 )
Tro ninfusiontherap

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM , pipeline , AutoModel , AutoModelForSeq2SeqLM
import torch
from langchain_huggingface.llms import HuggingFacePipeline

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it",device_map = 'auto')
pipe = pipeline("text-generation", model="google/gemma-2-2b-it",max_length = 7000,truncation = True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser , StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import Document
from langchain_huggingface import HuggingFacePipeline
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-002",api_key=GOOGLE_API_KEY,max_tokens=8000)
# Define the prompt template with JSON structure
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("human", """
        You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
        If information is missing or unknown, fill the corresponding field with "null". Ensure all fields are filled, even if no information is available.

        Question: {question}
        Context: {context}
        Answer:

        Format the response as a JSON object with the following structure, filling all fields and setting "null" for any missing information:

        {{
            "data": {{
                "transactionId": "transaction_id" ,
                "transactionType": "transaction_type",
                "hospital": {{
                    "name": "hospital_name" ,
                    "registration_number": ,
                    "address": "hospital_address" ,
                    "city": "hospital_city" ,
                    "state": "hospital_state" ,
                    "pin": "hospital_pin" ,
                    "email": "hospital_email" ,
                    "phone": "hospital_phone"
                }},
                "doctor": {{
                    "name": "doctor_name",
                    "registration_number": ,
                    "department": "doctor_department" ,
                    "designation": "doctor_designation" ,
                    "qualification": "doctor_qualification" ,
                    "email": "doctor_email" ,
                    "phone": "doctor_phone"
                }},
                "patient": {{
                    "name": "patient_name" ,
                    "dob": "patient_dob" ,
                    "address": "patient_address" ,
                    "city": "patient_city" ,
                    "state": "patient_state" ,
                    "pin": "patient_pin" ,
                    "tpa_id": "patient_tpa_id" ,
                    "employee_id": "employee_id" ,
                    "company_name": "company_name",
                    "pan_card": "pan_card" ,
                    "aadhar_card": "aadhar_card" ,
                    "phone": "patient_phone" ,
                    "email": "patient_email"
                }},
                "claim": [
                    {{
                        "doa": "claim_doa" ,
                        "dod": "claim_dod" ,
                        "total_claimed_amount": "claim_amount"
                    }},
                    {{
                        "invoice_date": "invoice_date_1" ,
                        "bill_group": "bill_group_1" ,
                        "bill_category": "bill_category_1" ,
                        "amount": "amount_1" ,
                        "date": "bill_date_1" ,
                        "bill_number": "bill_number_1" ,
                        "discount": "discount_1" ,
                        "quantity": "quantity_1",
                        "remarks": "remarks_1"
                    }},
                    {{
                        "invoice_date": "invoice_date_2" ,
                        "bill_group": "bill_group_2" ,
                        "bill_category": "bill_category_2" ,
                        "amount": "amount_2" ,
                        "date": "bill_date_2" ,
                        "bill_number": "bill_number_2" ,
                        "discount": "discount_2" ,
                        "quantity": "quantity_2" ,
                        "remarks": "remarks_2"
                    }}
                ],
                "medicalDetails": {{
                    "dischargeType": "discharge_type" ,
                    "patientAddress": {{
                        "address": "patient_address_line" ,
                        "city": "patient_city" ,
                        "state": "patient_state" ,
                        "pinCode": "patient_pin_code"
                    }},
                    "diagnosis": [
                        {{
                            "name": "diagnosis_name_1" ,
                            "icd": "diagnosis_icd_1"
                        }},
                        {{
                            "name": "diagnosis_name_2" ,
                            "icd": "diagnosis_icd_2"
                        }}
                    ],
                    "symptoms": [
                        "symptom_1" ,
                        "symptom_2" ,
                        "symptom_3"
                    ],
                    "treatment_taken": "treatment_taken"
                }},
                "room_invoice": [
                    {{
                        "room_type": "room_type" ,
                        "room_category": "room_category" ,
                        "start_date": "room_start_date" ,
                        "end_date": "room_end_date",
                        "rate": "room_rate",
                        "quantity": "room_quantity" ,
                        "discount": "room_discount"
                    }}
                ]
            }},
            "message": "Success",
            "success": true
        }}
    """)
])


# Define the function to process the question and context
def answer_with_json_format(format_docs, question):
    # Prepare the chain
    rag_chain = (
        {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
        | prompt
        | llm
        | JsonOutputParser()
    )

    # Execute the chain
    return rag_chain.invoke({"context": format_docs, "question": question})

# Convert `text_with_easy_ocr` to list of Document objects
documents = [Document(page_content=text_with_easy_ocr, metadata={"source": "/content/medical_report.pdf"})]

# Example usage
user_input = input("Please enter your question: ")
response_json = answer_with_json_format(format_docs=documents, question=user_input)

print(response_json)


Please enter your question: give all info upon Vikram Singh
{'data': {'transactionId': None, 'transactionType': None, 'hospital': {'name': 'City Care Hospital', 'registration_number': None, 'address': '123, Main Road, Delhi, 110001', 'city': 'Delhi', 'state': None, 'pin': '110001', 'email': 'info@citycarehospital.com', 'phone': '+91 11 2345 6789'}, 'doctor': {'name': 'Dr: Priya Mehta', 'registration_number': 'MCI-1234567', 'department': 'Nephrology', 'designation': 'Consultant Nephrologist', 'qualification': None, 'email': None, 'phone': None}, 'patient': {'name': 'Vikram Singh', 'dob': 'March 15, 1964', 'address': '45, Green Park Colony, Delhi, 110016', 'city': 'Delhi', 'state': None, 'pin': '110016', 'tpa_id': None, 'employee_id': None, 'company_name': 'ABC Technologies Pvt: Ltd.', 'pan_card': 'ABCPS1234E', 'aadhar_card': '1234-5678-9012', 'phone': None, 'email': None}, 'claim': [{'doa': 'October 10,2024', 'dod': None, 'total_claimed_amount': '69,900'}, {'invoice_date': None, 'bill_g

In [40]:
import json

with open('response_output.json', 'w') as file:
    json.dump(response_json, file, indent=4)

In [ ]:
import torch
torch.cuda.empty_cache()
